<a href="https://colab.research.google.com/github/AMJAMAITHILI/ML_LAB-141/blob/main/ML_LAB8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Comparing Single vs. Ensemble Classifiers

In [3]:
from google.colab import files
f=files.upload()

Saving diabetes.csv to diabetes.csv


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
df = pd.read_csv('diabetes.csv')
print(df.isnull().sum())
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [7]:
dt = DecisionTreeClassifier(random_state=42)#same splits and trees every time you run your code.
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
rf = RandomForestClassifier(random_state=42)#default 100 estimators
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
def print_metrics(y_true, y_pred, model_name):
    print(model_name+" Performance")
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"Recall:    {recall_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"F1 Score:  {f1_score(y_true, y_pred, zero_division=0):.4f}\n")
print_metrics(y_test, y_pred_dt, "Decision Tree")
print_metrics(y_test, y_pred_rf, "Random Forest")


Decision Tree Performance
Accuracy:  0.7013
Precision: 0.5556
Recall:    0.6875
F1 Score:  0.6145

Random Forest Performance
Accuracy:  0.7532
Precision: 0.6386
Recall:    0.6625
F1 Score:  0.6503



comparing single method with ensemble method->in case of Decision Tree evaluation metric<Random Forest except for the recall part The ensemble may prioritize better precision, reducing false positives, which can come at a minor cost to recall.Random Forest generally aims to reduce variance and overfitting by averaging multiple trees.

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}
rf = RandomForestClassifier(random_state=42)
scorer = make_scorer(accuracy_score)
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=scorer,
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)
print("Best parameters found: ", grid_search.best_params_)
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

print("\nTest set performance with best parameters:")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, zero_division=0):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred, zero_division=0):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred, zero_division=0):.4f}")


Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
540 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
394 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/uti

Best parameters found:  {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best cross-validation accuracy: 0.7784

Test set performance with best parameters:
Accuracy:  0.7532
Precision: 0.6420
Recall:    0.6500
F1 Score:  0.6460


In [13]:
n_estimators_list = [1,5,10, 50, 100, 200,300]

accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for n in n_estimators_list:
    rf = RandomForestClassifier(n_estimators=n, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred, zero_division=0))#zero_division=0->incase of zero division error
    recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
    f1_scores.append(f1_score(y_test, y_pred, zero_division=0))
for i in range(len(n_estimators_list)):
    print("n_estimator",n_estimators_list[i])
    print(f"Accuracy:  {accuracy_scores[i]:.4f}")
    print(f"Precision: {precision_scores[i]:.4f}")
    print(f"Recall:    {recall_scores[i]:.4f}")
    print(f"F1 Score:  {f1_scores[i]:.4f}\n")


n_estimator 1
Accuracy:  0.6667
Precision: 0.5161
Recall:    0.6000
F1 Score:  0.5549

n_estimator 5
Accuracy:  0.7186
Precision: 0.5862
Recall:    0.6375
F1 Score:  0.6108

n_estimator 10
Accuracy:  0.7403
Precision: 0.6316
Recall:    0.6000
F1 Score:  0.6154

n_estimator 50
Accuracy:  0.7403
Precision: 0.6163
Recall:    0.6625
F1 Score:  0.6386

n_estimator 100
Accuracy:  0.7532
Precision: 0.6386
Recall:    0.6625
F1 Score:  0.6503

n_estimator 200
Accuracy:  0.7532
Precision: 0.6420
Recall:    0.6500
F1 Score:  0.6460

n_estimator 300
Accuracy:  0.7532
Precision: 0.6420
Recall:    0.6500
F1 Score:  0.6460



increasing the no of estimator increases the evaluation metrics upto a point and there after taking 100 it decreased and remained constant(precision,recall,f1 score) gives minimal improvement so 100 suits the best

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from scipy.stats import mode
df = pd.read_csv('diabetes.csv')
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'KNN': KNeighborsClassifier()
}
predictions = {}
probabilities = {}
model_scores = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    prob = model.predict_proba(X_test)[:, 1]#of correctly predicted only there are 2 cols->-ve,+ve)

    predictions[name] = pred
    probabilities[name] = prob

    acc = accuracy_score(y_test, pred)
    model_scores[name] = acc
    print(f"{name} Accuracy: {acc:.4f}")

# 1. Max Voting (majority vote on predicted classes)
all_preds = np.array(list(predictions.values()))
max_vote_preds, _ = mode(all_preds, axis=0)
max_vote_preds = max_vote_preds.flatten()

# 2. Average Voting (average predicted probabilities, threshold=0.5)
avg_probs = np.mean(np.array(list(probabilities.values())), axis=0)
avg_vote_preds = (avg_probs >= 0.5).astype(int)

# 3. Weighted Average Voting (weights based on model accuracy)
weights = np.array([model_scores[name] for name in models.keys()])
weights = weights / weights.sum()  # normalize weights to sum=1

weighted_probs = np.zeros_like(avg_probs)
for i, name in enumerate(models.keys()):
    weighted_probs += weights[i] * probabilities[name]

weighted_vote_preds = (weighted_probs >= 0.5).astype(int)
"""array([
    [0.2, 0.8, 0.6, 0.4],  # Logistic Regression
    [0.3, 0.7, 0.9, 0.2],  # Decision Tree
    [0.1, 0.9, 0.8, 0.5]   # KNN
])
avg1​=(0.2+0.3+0.1)/3=0.2avg2​=(0.8+0.7+0.9)/3=0.8avg3​=(0.6+0.9+0.8)/3=0.7667avg4​=(0.4+0.2+0.5)/3=0.3667->avg voting
weight*prob->weighted_prob
"""

# Evaluation function
def print_metrics(y_true, y_pred, label):
    print(label)
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"Recall:    {recall_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"F1 Score:  {f1_score(y_true, y_pred, zero_division=0):.4f}\n")

# Print results
print_metrics(y_test, max_vote_preds, "Max Voting Ensemble")
print_metrics(y_test, avg_vote_preds, "Average Voting Ensemble")
print_metrics(y_test, weighted_vote_preds, "Weighted Average Voting Ensemble")


Decision Tree Accuracy: 0.7013
Logistic Regression Accuracy: 0.7359
KNN Accuracy: 0.6883
Max Voting Ensemble
Accuracy:  0.7186
Precision: 0.5862
Recall:    0.6375
F1 Score:  0.6108

Average Voting Ensemble
Accuracy:  0.7100
Precision: 0.5714
Recall:    0.6500
F1 Score:  0.6082

Weighted Average Voting Ensemble
Accuracy:  0.7100
Precision: 0.5714
Recall:    0.6500
F1 Score:  0.6082



| Type            | What It Does          | How It Works                                                                                                                          |
| --------------- | --------------------- | ------------------------------------------------------------------------------------------------------------------------------------- |
| **Hard Voting** | Majority Voting       | Each model predicts a class label (0 or 1). The final class is chosen by majority vote.                                               |
| **Soft Voting** | Probability Averaging | Each model outputs class probabilities. The probabilities are averaged, and the class with the highest average probability is chosen. |


In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model1 = LogisticRegression(max_iter=1000, random_state=42)
model2 = DecisionTreeClassifier(random_state=42)
model3 = KNeighborsClassifier()

hard_voting = VotingClassifier(
    estimators=[('lr', model1), ('dt', model2), ('knn', model3)],
    voting='hard'
)

soft_voting = VotingClassifier(
    estimators=[('lr', model1), ('dt', model2), ('knn', model3)],
    voting='soft'
)

hard_voting.fit(X_train, y_train)
soft_voting.fit(X_train, y_train)

y_pred_hard = hard_voting.predict(X_test)
y_pred_soft = soft_voting.predict(X_test)

print("Hard Voting Performance:")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_hard):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_hard, zero_division=0):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_hard, zero_division=0):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_hard, zero_division=0):.4f}")

print("\nSoft Voting Performance:")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_soft):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_soft, zero_division=0):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_soft, zero_division=0):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_soft, zero_division=0):.4f}")


Hard Voting Performance:
Accuracy:  0.7186
Precision: 0.5862
Recall:    0.6375
F1 Score:  0.6108

Soft Voting Performance:
Accuracy:  0.7100
Precision: 0.5714
Recall:    0.6500
F1 Score:  0.6082
